In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.lines as mlines
import os
import io
import requests

In [3]:
def load_and_group_dataframe(shareable_link):
    """
    Loads a JSON file from a Google Drive shareable link into a pandas DataFrame
    and displays the head of the DataFrame with columns grouped by numerical
    and then non-numerical data types.

    Args:
        shareable_link (str): The "Anyone with the link can view" link to the
                              JSON file on Google Drive.
    """
    file_id = None
    if "/d/" in shareable_link and "/view" in shareable_link:
        file_id = shareable_link.split("/d/")[1].split("/view")[0]
    elif "/file/d/" in shareable_link and "/edit" in shareable_link:
        file_id = shareable_link.split("/file/d/")[1].split("/edit")[0]

    if file_id:
        download_url = f"https://drive.google.com/uc?id={file_id}&export=download"

        try:
            response = requests.get(download_url)
            response.raise_for_status()  # Raise an exception for bad status codes

            # Read the JSON data directly from the response content
            ut_data = pd.read_json(io.StringIO(response.text))
            print("Data loaded successfully from Google Drive:")

            # Separate numerical and non-numerical columns
            numerical_cols = ut_data.select_dtypes(include=['number']).columns.tolist()
            non_numerical_cols = ut_data.select_dtypes(exclude=['number']).columns.tolist()

            # Create the desired column order
            ordered_cols = numerical_cols + non_numerical_cols

            # Display the DataFrame with the ordered columns (showing the head)
            #print(ut_data[ordered_cols].head())

            return ut_data  # Return the loaded DataFrame

        except requests.exceptions.RequestException as e:
            print(f"Error downloading file from Google Drive: {e}")
            return None
        except pd.errors.EmptyDataError:
            print("Error: The downloaded file appears to be empty.")
            return None
        except Exception as e:
            print(f"An error occurred while processing the downloaded data: {e}")
            return None

    else:
        print("Error: Could not extract file ID from the shareable link. Please ensure the link is correct.")
        return None

In [ ]:
def find_n_plus_duplicate_timestamps(df, column_name='login_time', n=3):
    """
    Finds 'login timestamp' values that appear n or more times in the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to check.
        column_name (str): The name of the timestamp column. Defaults to 'login timestamp'.
        n (int): The minimum number of times a timestamp must be duplicated.
                 Defaults to 3.

    Returns:
        pd.DataFrame: A DataFrame containing all rows where the 'login timestamp'
                      appears n or more times.
    """
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the DataFrame.")
        return pd.DataFrame()

    timestamp_counts = df[column_name].value_counts()
    n_plus_duplicate_timestamps = timestamp_counts[timestamp_counts >= n].index.tolist()

    if n_plus_duplicate_timestamps:
        n_plus_duplicates_df = df[df[column_name].isin(n_plus_duplicate_timestamps)]
        return n_plus_duplicates_df.sort_values(by=column_name)
    else:
        print(f"No 'login timestamp' values appear {n} or more times.")
        return pd.DataFrame()

### Part 1

In [12]:
# Example usage:
shareable_link = "https://drive.google.com/file/d/1ZA1-IVF15jD_IIIxCQ6a8dEG94Qf9raz/view?usp=drive_link"
utl_data = load_and_group_dataframe(shareable_link)

# You can now work with the loaded DataFrame 'df' if it was loaded successfully
if utl_data is not None:
    # Further analysis or processing
    print("\nDataFrame shape:", utl_data.shape)

utl_data.head()

Data loaded successfully from Google Drive:

DataFrame shape: (93142, 1)


,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37
3,1970-01-01 20:16:36
4,1970-01-01 20:26:21


In [6]:
utl_data.isnull().sum()

login_time    0
dtype: int64

In [17]:
# Checking for Duplicate Rows (Based on All Columns):

# Check for duplicate rows
duplicate_rows = utl_data[utl_data.duplicated()]

# Print the duplicate rows
#print("Duplicate Rows (based on all columns):")
#print(duplicate_rows)

# Get the number of duplicate rows
num_duplicate_rows = duplicate_rows.shape[0]
print(f"\nNumber of duplicate rows: {num_duplicate_rows}")

# To see all unique rows (removing duplicates):
utl_data_unique_rows = utl_data.drop_duplicates()
print("\nDataFrame with duplicate rows removed:")
print(f"Shape of DataFrame with duplicates removed: {utl_data_unique_rows.shape}")
utl_data_unique_rows.head()



Number of duplicate rows: 877

DataFrame with duplicate rows removed:
Shape of DataFrame with duplicates removed: (92265, 1)


,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37
3,1970-01-01 20:16:36
4,1970-01-01 20:26:21


In [14]:
# Find login timestamps that appear 3 or more times:
n_plus_duplicate_timestamps_df = find_n_plus_duplicate_timestamps(utl_data, n=3)
print("\nLogin timestamps appearing 3 or more times:")
print(n_plus_duplicate_timestamps_df)


Login timestamps appearing 3 or more times:
               login_time
3253  1970-01-06 21:45:52
3256  1970-01-06 21:45:52
3259  1970-01-06 21:45:52
4744  1970-01-09 01:31:25
4750  1970-01-09 01:31:25
4757  1970-01-09 01:31:25
29774 1970-02-12 11:16:53
29790 1970-02-12 11:16:53
29797 1970-02-12 11:16:53
41874 1970-02-24 23:09:57
41877 1970-02-24 23:09:57
41883 1970-02-24 23:09:57
59819 1970-03-14 17:45:52
59806 1970-03-14 17:45:52
59791 1970-03-14 17:45:52
62890 1970-03-17 01:01:39
62902 1970-03-17 01:01:39
62908 1970-03-17 01:01:39
77076 1970-03-29 03:24:57
77094 1970-03-29 03:24:57
77096 1970-03-29 03:24:57
83421 1970-04-04 01:17:40
83458 1970-04-04 01:17:40
83478 1970-04-04 01:17:40
91843 1970-04-12 04:03:15
91862 1970-04-12 04:03:15
91869 1970-04-12 04:03:15


In [ ]:
Is it possible for users to log in at the exact same time down to the milliseconds? If yes, then we need to keep the data intact and not drop duplicates. 

### Part 2

In [ ]:
# Example usage:
shareable_link = "https://drive.google.com/file/d/1buEu5LHKSuK0Pgyu4cpYClOgDnlzdnJY/view?usp=drive_link"
ut_data = load_and_group_dataframe(shareable_link)

# You can now work with the loaded DataFrame 'df' if it was loaded successfully
if ut_data is not None:
    # Further analysis or processing
    print("\nDataFrame shape:", ut_data.shape)

In [ ]:
ut_data.shape

In [ ]:
ut_data.describe()

In [ ]:
# Separate numerical and non-numerical columns
numerical_cols = ut_data.select_dtypes(include=['number']).columns.tolist()
non_numerical_cols = ut_data.select_dtypes(exclude=['number']).columns.tolist()

# Create the desired column order
ordered_cols = numerical_cols + non_numerical_cols

# Display the DataFrame with the ordered columns (showing the head)
ut_data[ordered_cols].head()

# If you want to see all columns without truncation in the output:
# pd.set_option('display.max_columns', None)
# print(ut_data[ordered_cols])